Before importing the data from the previous step and the packages we need, we list some of the things we noted in the previous step which we need to deal with:
1. We need to convert some of features into the correct data type.
2. We have up to 108 rows with missing data on the number of half baths.
3. There are some properties with very high acreage and/or finished area but with low sale prices.
4. Some of the sale prices are incredibly low and bedroom/bathroom numbers for some properties are zero.
5. There might be duplicate neighborhood IDs for different cities.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

In [2]:
df = pd.read_csv('Nashville_Housing_Prices_EDAcompleted.csv', index_col=0)

In [3]:
df.head()

,Parcel ID,Land Use,Property Address,Property City,Sale Date,Sale Price,Legal Reference,Sold As Vacant,Multiple Parcels Involved in Sale,State,...,Grade,Bedrooms,Full Bath,Half Bath,Month,Year,US Unemployment Rate,Nashville Unemployment Rate,US Average Hourly Wage,Mortgage Rate
1,091 07 0 171.00,SINGLE FAMILY,4806 KENTUCKY AVE,NASHVILLE,2013-01-02,50000,20130103-0001026,False,False,TN,...,D,3.0,1.0,0.0,Jan,2013,8.0,6.8,19.94,3.34
3,105 13 0 349.00,SINGLE FAMILY,924 WALDKIRCH AVE,NASHVILLE,2013-01-02,225000,20130107-0001749,False,False,TN,...,C,2.0,1.0,0.0,Jan,2013,8.0,6.8,19.94,3.34
4,136 14 0 036.00,SINGLE FAMILY,525 CASTLEGATE DR,NASHVILLE,2013-01-02,152000,20130110-0003190,False,False,TN,...,C,4.0,3.0,0.0,Jan,2013,8.0,6.8,19.94,3.34
6,159 00 0 261.00,SINGLE FAMILY,5442 GRANNY WHITE PIKE,BRENTWOOD,2013-01-02,1365000,20130104-0001158,False,False,TN,...,X,4.0,4.0,1.0,Jan,2013,8.0,6.8,19.94,3.34
9,075 11 0 189.00,ZERO LOT LINE,813 BRADLEY PL,HERMITAGE,2013-01-03,47025,20130128-0008570,False,False,TN,...,C,3.0,1.0,1.0,Jan,2013,8.0,6.8,19.94,3.34


We first deal with any missing data.

In [4]:
df.isna().sum()

Parcel ID                              0
Land Use                               0
Property Address                       0
Property City                          0
Sale Date                              0
Sale Price                             0
Legal Reference                        0
Sold As Vacant                         0
Multiple Parcels Involved in Sale      0
State                                  0
Acreage                                0
Tax District                           0
Neighborhood                           0
Finished Area                          0
Foundation Type                        0
Year Built                             0
Exterior Wall                          0
Grade                                  0
Bedrooms                               0
Full Bath                              0
Half Bath                            103
Month                                  0
Year                                   0
US Unemployment Rate                   0
Nashville Unempl

We have 103 properties with missing data on the number of half baths. It seems plausible to assume that these values are actually zero, so we impute the missing valuues with zeros.

In [5]:
df.loc[df['Half Bath'].isna(),'Half Bath'] = 0

We then look into the properties with very low sale prices.

In [6]:
df[df['Sale Price']/df['Finished Area']<50].head()

,Parcel ID,Land Use,Property Address,Property City,Sale Date,Sale Price,Legal Reference,Sold As Vacant,Multiple Parcels Involved in Sale,State,...,Grade,Bedrooms,Full Bath,Half Bath,Month,Year,US Unemployment Rate,Nashville Unemployment Rate,US Average Hourly Wage,Mortgage Rate
9,075 11 0 189.00,ZERO LOT LINE,813 BRADLEY PL,HERMITAGE,2013-01-03,47025,20130128-0008570,False,False,TN,...,C,3.0,1.0,1.0,Jan,2013,8.0,6.8,19.94,3.34
17,041 11 0 011.00,SINGLE FAMILY,4021 DUMBARTON DR,NASHVILLE,2013-01-04,100,20130129-0009438,False,False,TN,...,C,2.0,1.0,0.0,Jan,2013,8.0,6.8,19.94,3.34
18,080 08 0 260.00,SINGLE FAMILY,3234 MEXICO DR,NASHVILLE,2013-01-04,20000,20130107-0001791,False,False,TN,...,C,3.0,1.0,0.0,Jan,2013,8.0,6.8,19.94,3.34
21,061 11 0 235.00,SINGLE FAMILY,1105 GREENLAND AVE,NASHVILLE,2013-01-04,83000,20130110-0003212,False,False,TN,...,C,4.0,3.0,0.0,Jan,2013,8.0,6.8,19.94,3.34
43,072 10 0 314.00,SINGLE FAMILY,1640 CHASE ST,NASHVILLE,2013-01-07,53000,20130108-0002420,False,False,TN,...,C,3.0,2.0,1.0,Jan,2013,8.0,6.8,19.94,3.40


We see that there are properties with a sale price of less than \\$50 per square foot of finished area. This is very low and probably asscoiated with errors. In fact, a comprehensive search on the interent showed that currently houses in Nashville metro area sell on average for a price of \\$165-\\$303 per square foot. Since the prices from the period of analysis (2013-2016) to present have almost doubled, it is plausible to assume that the median price per square foot for the period of analysis should be around \\$100-\\$150. We however, take a conservative approach and assume only the prices outside of the range of \\$50-\\$350 are associated with errors. This approach aslo addresses the problem of properties with very high acreage and/or finished area and low sale prices.

In [7]:
print(df[(df['Sale Price']/df['Finished Area']<50) | (df['Sale Price']/df['Finished Area']>350)].shape)
df = df.drop(index = df[(df['Sale Price']/df['Finished Area']<50) | \
                        (df['Sale Price']/df['Finished Area']>350)].index)

(1462, 27)


We see there were 1462 rows asscoiated with these properties which have been dropped.

For properties with zero bathrooms/bedrooms, we impute the number of bathrooms/bedrooms with rounded average number of bathrooms/bedrooms for properties with similar finished areas.

In [8]:
temp1 = pd.DataFrame(round(df.loc[df['Full Bath'] == 0,'Finished Area']/100)*100)
temp2 = pd.DataFrame(round(df.loc[df['Bedrooms'] == 0,'Finished Area']/100)*100)

In [9]:
for index in temp1.index:
    df.loc[index,'Full Bath'] = round(df.loc[(df['Finished Area']>temp1.loc[index,'Finished Area']-500)&(df['Finished Area']<temp1.loc[index,'Finished Area']+500),'Full Bath'].mean())
for index in temp2.index:
    df.loc[index,'Bedrooms'] = round(df.loc[(df['Finished Area']>temp2.loc[index,'Finished Area']-500)&(df['Finished Area']<temp2.loc[index,'Finished Area']+500),'Bedrooms'].mean())

We then convert the features into correct data types. We also combine some of the features into one categorical feature.

In [10]:
df.dtypes

Parcel ID                             object
Land Use                              object
Property Address                      object
Property City                         object
Sale Date                             object
Sale Price                             int64
Legal Reference                       object
Sold As Vacant                          bool
Multiple Parcels Involved in Sale       bool
State                                 object
Acreage                              float64
Tax District                          object
Neighborhood                         float64
Finished Area                        float64
Foundation Type                       object
Year Built                           float64
Exterior Wall                         object
Grade                                 object
Bedrooms                             float64
Full Bath                            float64
Half Bath                            float64
Month                                 object
Year      

In [11]:
df['Bedrooms'] = df['Bedrooms'].astype('int')
df['Full Bath'] = df['Full Bath'].astype('int')
df['Half Bath'] = df['Half Bath'].astype('int')
df['Neighborhood'] = df['Neighborhood'].astype('int')

In [12]:
df['Location'] = df['Property City'] + ' ' + df['Neighborhood'].astype('str')

In [13]:
df['Sale Month/Year'] = df['Month'] + ' ' + df['Year'].astype('str')

In [14]:
df['Age when sold'] = df['Year'] - df['Year Built']

We now perform one-hot encoding for categorical variables, scale our data, and split into train and test datasets.

In [15]:
X = df[['Land Use', 'Sold As Vacant', 'Multiple Parcels Involved in Sale', 'Acreage', 'Tax District',
       'Finished Area', 'Foundation Type', 'Exterior Wall', 'Grade', 'Bedrooms', 'Full Bath', 'Half Bath',
       'Nashville Unemployment Rate', 'US Average Hourly Wage', 'Mortgage Rate', 'Location',
       'Sale Month/Year', 'Age when sold']].reset_index(drop=True)
y=df[['Sale Price']].reset_index(drop=True)

In [16]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_columns = pd.DataFrame(scaler.fit_transform(X[['Acreage', 'Finished Area', 'Bedrooms', 'Full Bath', 'Half Bath',
       'Nashville Unemployment Rate', 'US Average Hourly Wage', 'Mortgage Rate', 'Age when sold']]), 
        columns=['Acreage', 'Finished Area', 'Bedrooms', 'Full Bath', 'Half Bath',
       'Nashville Unemployment Rate', 'US Average Hourly Wage', 'Mortgage Rate', 'Age when sold'])
encoded_columns = pd.get_dummies(X[['Land Use', 'Tax District', 'Foundation Type', 'Exterior Wall', 
       'Grade', 'Location', 'Sale Month/Year']], drop_first=True)

In [17]:
Xtemp = pd.concat([scaled_columns, encoded_columns], axis=1)
X = pd.concat([Xtemp, df[['Sold As Vacant', 'Multiple Parcels Involved in Sale']].reset_index(drop=True)], axis=1)

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [19]:
y_train.shape

(16377, 1)

We save the data into a CSV file for modelling in the next step.

In [20]:
X_train.to_csv('X_train.csv')
X_test.to_csv('X_test.csv')
y_train.to_csv('y_train.csv')
y_test.to_csv('y_test.csv')